Modèle de réseau de neurones sur le dataset du Joueur bas - Henri, Minh, Maxime
- Coup droit
- Déplacement
- Revers
- Service

In [46]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder

In [47]:
# Configuration
path_dataset = 'img/JBas'

# Réseau de neurones classique

## Prises des frames et des vidéos

In [48]:
list_videos = []

In [49]:
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [50]:
list_videos

In [51]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup dro

In [52]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [53]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [54]:
output_y = np.unique(y)
output_y

array([0, 1, 2, 3], dtype=int64)

In [55]:
# Résolution des images
input_shape_model = list_videos[0].frames.shape[0] * list_videos[0].frames.shape[1] * list_videos[0].frames.shape[2]
input_shape_model

37500

In [56]:
len(list_videos)

214

In [57]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [58]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(171, 171, 43, 43)

In [59]:
paths_test

['img/JBas\\coup droit\\1980-2563441\\images',
 'img/JBas\\coup droit\\1980-2563523\\images',
 'img/JBas\\coup droit\\42698-4288095\\images',
 'img/JBas\\coup droit\\a36r805\\images',
 'img/JBas\\coup droit\\Alcaraz_Sinner270\\images',
 'img/JBas\\coup droit\\AS_13416\\images',
 'img/JBas\\coup droit\\AS_22373\\images',
 'img/JBas\\coup droit\\AS_30502\\images',
 'img/JBas\\coup droit\\AS_37288\\images',
 'img/JBas\\coup droit\\AS_40159\\images',
 'img/JBas\\coup droit\\car0001-0342253\\images',
 'img/JBas\\coup droit\\car5493-5847185\\images',
 'img/JBas\\coup droit\\na0201977\\images',
 'img/JBas\\coup droit\\uopc1041\\images',
 'img/JBas\\coup droit\\uopc1683\\images',
 'img/JBas\\coup droit\\uopc339\\images',
 'img/JBas\\deplacement\\a36r1216\\images',
 'img/JBas\\deplacement\\a36r1713\\images',
 'img/JBas\\deplacement\\a36r172\\images',
 'img/JBas\\deplacement\\a36r257\\images',
 'img/JBas\\deplacement\\a36r339\\images',
 'img/JBas\\deplacement\\AS_13292\\images',
 'img/JBas\\depl

In [60]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [61]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [62]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [63]:
# Normalisation
X_train = X_train / 255.0
X_test = X_test / 255.0

## Modèle de réseau de neurones avec classification multiclass

In [64]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        tf.keras.Input(shape = input_shape_model), # 50*50 * 15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=64, activation="sigmoid"),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)          
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 16)                600016    
                                                                 
 dense_8 (Dense)             (None, 32)                544       
                                                                 
 dense_9 (Dense)             (None, 64)                2112      
                                                                 
 dense_10 (Dense)            (None, 128)               8320      
                                                                 
 dense_11 (Dense)            (None, 4)                 516       
                                                                 
Total params: 611,508
Trainable params: 611,508
Non-trainable params: 0
_________________________________________________________________


In [65]:
checkpoint_name = 'checkpoints/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [66]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs = 70,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
1/5 [=====>........................] - ETA: 0s - loss: 1.4128 - accuracy: 0.3750
Epoch 1: val_loss improved from inf to 2.71367, saving model to checkpoints\Weights-001--2.71367.hdf5
5/5 [==============================] - 0s 41ms/step - loss: 1.2540 - accuracy: 0.3897 - val_loss: 2.7137 - val_accuracy: 0.0000e+00
Epoch 2/70
1/5 [=====>........................] - ETA: 0s - loss: 1.0991 - accuracy: 0.3438
Epoch 2: val_loss did not improve from 2.71367
5/5 [==============================] - 0s 10ms/step - loss: 1.0726 - accuracy: 0.4853 - val_loss: 3.5563 - val_accuracy: 0.0000e+00
Epoch 3/70
1/5 [=====>........................] - ETA: 0s - loss: 1.0235 - accuracy: 0.4688
Epoch 3: val_loss did not improve from 2.71367
5/5 [==============================] - 0s 10ms/step - loss: 0.9866 - accuracy: 0.4632 - val_loss: 4.0729 - val_accuracy: 0.0000e+00
Epoch 4/70
1/5 [=====>........................] - ETA: 0s - loss: 0.9764 - accuracy: 0.4375
Epoch 4: val_loss did not improve from 2

In [67]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 3.7356 - accuracy: 0.3488


[3.73563551902771, 0.3488371968269348]

In [68]:
pred = model.predict(X_test)
pred

2/2 [==============================] - 0s 2ms/step


array([[3.31466552e-03, 9.80816066e-01, 1.58282015e-02, 4.09925342e-05],
       [1.35716632e-01, 3.96943390e-02, 8.24380934e-01, 2.08058686e-04],
       [1.80883519e-02, 9.81426179e-01, 4.58119554e-04, 2.73225451e-05],
       [2.36354047e-03, 9.97216702e-01, 4.06670762e-04, 1.30653252e-05],
       [9.60909296e-03, 9.87393260e-01, 2.95358594e-03, 4.40086988e-05],
       [1.24936458e-02, 9.75139022e-01, 1.23040443e-02, 6.33372983e-05],
       [8.57563674e-01, 9.75261815e-03, 1.32515788e-01, 1.67973907e-04],
       [9.99868155e-01, 1.04768485e-04, 2.35154621e-05, 3.62156675e-06],
       [1.63514761e-03, 9.91771996e-01, 6.56424277e-03, 2.86319755e-05],
       [8.57678428e-02, 9.13446844e-01, 7.34232424e-04, 5.10491082e-05],
       [9.72295284e-01, 8.00719298e-03, 1.96219534e-02, 7.55474393e-05],
       [9.99745905e-01, 2.15173990e-04, 3.46444904e-05, 4.21551840e-06],
       [2.72275734e-04, 9.99612033e-01, 1.11248366e-04, 4.55079817e-06],
       [6.20603681e-01, 2.95015305e-01, 8.40738118e

In [24]:
y_pred = np.argmax(pred, axis=1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  [1 2 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 1 1 0 1 0 2 0 1 1 0 0 0 0 1 0 2 2 0 0 0
 0 1 0 1 1 0]
True labels:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 3 3 3 3]


**Accuracy: $17 / 43$ ($39.53 \%$)**

# Réseaux de neurones convolutifs

## Prises des frames et des vidéos

In [25]:
list_videos_cnn = []
y_cnn = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_COLOR, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    vid = Video.read_video_from_frames(frames)
                    list_videos_cnn.append(vid)
                    y_cnn.append(output_res)

In [26]:
list_videos_cnn[0].frames.shape[1:]

(50, 50, 3)

In [27]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [28]:
y_cnn = encoder.fit_transform(y_cnn)
y_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [29]:
output_y = np.unique(y_cnn)
output_y

array([0, 1, 2, 3], dtype=int64)

In [30]:
len(list_videos_cnn)

214

In [31]:
train_size = int(len(list_videos_cnn) * 0.8)
test_size = len(list_videos_cnn) - train_size

In [32]:
## Split video train, test
random.seed(42)
vids_train_cnn = random.sample(list_videos_cnn, k = train_size)
vids_test_cnn = [vid for vid in list_videos_cnn if vid not in vids_train_cnn]
y_train_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_train_cnn]
y_test_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_test_cnn]
len(vids_train_cnn), len(y_train_cnn), len(vids_test_cnn), len(y_test_cnn)

(171, 171, 43, 43)

In [33]:
y_train_cnn = np.array(y_train_cnn, dtype=int)
y_test_cnn = np.array(y_test_cnn, dtype=int)
y_train_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [34]:
# Split input X en train, test set
X_train_cnn = np.zeros((train_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_train_cnn):
    for j, frame in enumerate(vid.frames):
        X_train_cnn[i, :, 50*j:50*(j+1), :] = frame
X_train_cnn.shape

(171, 50, 750, 3)

In [35]:
X_test_cnn = np.zeros((test_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_test_cnn):
    for j, frame in enumerate(vid.frames):
        X_test_cnn[i, :, 50*j:50*(j+1), :] = frame
X_test_cnn.shape

(43, 50, 750, 3)

In [36]:
input_shape_cnn = X_train_cnn.shape[1:]
input_shape_cnn

(50, 750, 3)

## Modèle de réseau de neurones convolutifs

In [37]:
model_cnn = Sequential(
    [
        tf.keras.Input(shape = input_shape_cnn), # 50, 15*50, 3
        Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)

model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 748, 16)       448       
                                                                 
 batch_normalization (BatchN  (None, 48, 748, 16)      64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 374, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 374, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 372, 32)       4640      
                                                                 
 batch_normalization_1 (Batc  (None, 22, 372, 32)     

In [40]:
checkpoint_name = 'checkpoints/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [41]:
model_cnn.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer='rmsprop',
    metrics=['accuracy']
)

history_cnn = model_cnn.fit(
    X_train_cnn, y_train_cnn,
    epochs = 70,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
5/5 [==============================] - ETA: 0s - loss: 7.7146 - accuracy: 0.7132
Epoch 1: val_loss improved from inf to 44.49462, saving model to checkpoints\Weights-001--44.49462.hdf5
5/5 [==============================] - 2s 282ms/step - loss: 7.7146 - accuracy: 0.7132 - val_loss: 44.4946 - val_accuracy: 0.0000e+00
Epoch 2/70
5/5 [==============================] - ETA: 0s - loss: 1.8054 - accuracy: 0.7721
Epoch 2: val_loss improved from 44.49462 to 37.66714, saving model to checkpoints\Weights-002--37.66714.hdf5
5/5 [==============================] - 1s 240ms/step - loss: 1.8054 - accuracy: 0.7721 - val_loss: 37.6671 - val_accuracy: 0.2857
Epoch 3/70
5/5 [==============================] - ETA: 0s - loss: 0.2365 - accuracy: 0.9265
Epoch 3: val_loss improved from 37.66714 to 28.19355, saving model to checkpoints\Weights-003--28.19355.hdf5
5/5 [==============================] - 1s 231ms/step - loss: 0.2365 - accuracy: 0.9265 - val_loss: 28.1935 - val_accuracy: 0.1143
Epoch 4/

In [42]:
model_cnn.evaluate(X_test_cnn, y_test_cnn)

2/2 [==============================] - 0s 68ms/step - loss: 12.5316 - accuracy: 0.4651


[12.531560897827148, 0.4651162922382355]

In [44]:
pred_cnn = model_cnn.predict(X_test_cnn)
pred_cnn

2/2 [==============================] - 0s 46ms/step


array([[6.10227513e-09, 1.00000000e+00, 1.32505465e-12, 4.61441547e-17],
       [9.99992013e-01, 1.13003495e-07, 7.83554515e-06, 3.15865059e-25],
       [2.73185894e-02, 2.07257050e-04, 9.72474158e-01, 1.48067355e-25],
       [5.79412937e-01, 4.20587033e-01, 1.77708015e-09, 1.94915438e-21],
       [2.91187898e-04, 9.99708831e-01, 1.63423081e-21, 6.81038953e-35],
       [9.99757588e-01, 1.99009912e-04, 4.33617461e-05, 1.50830493e-36],
       [9.28058743e-01, 7.10723251e-02, 8.68952600e-04, 2.49300099e-23],
       [9.99992967e-01, 7.02123907e-06, 7.17429362e-17, 2.30603052e-27],
       [2.40731822e-07, 9.99999762e-01, 2.84012028e-14, 9.87735591e-30],
       [6.93466246e-01, 3.06484908e-01, 4.88719124e-05, 9.27294457e-25],
       [9.99943018e-01, 5.68023970e-05, 1.39527458e-07, 2.60141890e-26],
       [1.00000000e+00, 1.87313179e-10, 1.28742208e-11, 0.00000000e+00],
       [9.97323632e-01, 2.46249674e-05, 2.65174056e-03, 7.19965145e-20],
       [1.60385177e-01, 8.39614868e-01, 4.60369448e

In [45]:
y_pred_cnn = np.argmax(pred_cnn, axis=1)
print("Predicted labels: ", y_pred_cnn)
print("True labels:      ", y_test_cnn)

Predicted labels:  [1 0 2 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 2 1 1 1 2 1 0 1 1 1 0 0 0 0 1
 1 0 0 1 0 0]
True labels:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 3 3 3 3]


**Accuracy: $20/43$ ($46.51 \%$)**